### <b><mark style="background-color: #9B59B6"><font color="white">Background on the National Sex Offenders Registry:</font></mark></b>


The Pam Lychner Sexual Offender Tracking and Identification Act of 1996 (Lychner Act).

The Jacob Wetterling Crimes Against Children and Sexual Violent Offender Registration Program (1994).

Megan’s Law (1996).

https://www.fbi.gov/scams-and-safety/sex-offender-registry

![](https://www.jeffersoncountysheriff.com/wp-content/uploads/2019/03/sor.jpg)jeffersoncountysheriff.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/sex-offender-registry/Sex_Offender_Registry.csv', encoding='utf8')
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
#Codes by Pooja Jain https://www.kaggle.com/jainpooja/av-guided-hackathon-predict-youtube-likes/notebook

text_cols = ['ALIASES', 'MARKINGS', 'TYPE']

from wordcloud import WordCloud, STOPWORDS

wc = WordCloud(stopwords = set(list(STOPWORDS) + ['|']), random_state = 42)
fig, axes = plt.subplots(2, 2, figsize=(20, 12))
axes = [ax for axes_row in axes for ax in axes_row]

for i, c in enumerate(text_cols):
  op = wc.generate(str(df[c]))
  _ = axes[i].imshow(op)
  _ = axes[i].set_title(c.upper(), fontsize=24)
  _ = axes[i].axis('off')

_ = fig.delaxes(axes[3])

In [ ]:
# categorical features with missing values
categorical_nan = [feature for feature in df.columns if df[feature].isna().sum()>0 and df[feature].dtypes=='O']
print(categorical_nan)

In [ ]:
# replacing missing values in categorical features
for feature in categorical_nan:
    df[feature] = df[feature].fillna('None')

In [ ]:
df[categorical_nan].isna().sum()

#Codes by Arpita Gupta https://www.kaggle.com/arpita28/analysis-of-spotify-trends

In [ ]:
from scipy import stats

#Finding out the skew for each attribute
skew=df.skew()
print(skew)
# Removing the skew by using the boxcox transformations
transform=np.asarray(df[['X']].values)
df_transform = stats.boxcox(transform)[0]
#Plotting a histogram to show the difference 
plt.hist(df['X'],bins=10) #original data
plt.show()
plt.hist(df_transform,bins=10) #corrected skew data
plt.show()

In [ ]:
pd.set_option('display.width', 100)
pd.set_option('precision', 3)
correlation=df.corr(method='spearman')
print(correlation)

In [ ]:
# heatmap of the correlation 
plt.figure(figsize=(10,10))
plt.title('Correlation heatmap')
sns.heatmap(correlation,annot=True,vmin=-1,vmax=1,cmap="GnBu_r",center=1)

In [ ]:
fig = plt.figure(figsize = (15,7))
df.groupby('LASTNAME')['SEXOFFENDERCODE'].agg(len).sort_values(ascending = False).head(20).plot(kind = 'bar')
plt.xlabel('LASTNAME', fontsize = 20)
plt.ylabel('SEXOFFENDERCODE', fontsize = 20)
plt.title('20 Sex Offenders by Lastname and Code', fontsize = 30)

In [ ]:
# Analysing the relationship between energy and loudness
fig=plt.subplots(figsize=(10,10))
sns.regplot(x='X',y='Y',data=df,color='purple')

I don't know the meaning of X and Y, though the programm didn't return an error.

In [ ]:
fig=plt.subplots(figsize=(10,10))
plt.title('Dependence between X and Y')
sns.regplot(x='X', y='Y',
            ci=None, data=df)
sns.kdeplot(df.X,df.Y)

It's cool but I have no clue what that chart represents. It seems abstract art.

In [ ]:
df.plot(kind='box', subplots=True)
plt.gcf().set_size_inches(30,30)
plt.show()

In [ ]:
import squarify as sq

plt.figure(figsize=(14,8))
sq.plot(sizes=df.QUADRANT.value_counts(), label=df["QUADRANT"].unique(), alpha=.8 )
plt.axis('off')
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

#fill in mean for floats
for c in df.columns:
    if df[c].dtype=='float16' or  df[c].dtype=='float32' or  df[c].dtype=='float64':
        df[c].fillna(df[c].mean())

#fill in -999 for categoricals
df = df.fillna(-999)
# Label Encoding
for f in df.columns:
    if df[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(df[f].values))
        df[f] = lbl.transform(list(df[f].values))
        
print('Labelling done.')

In [ ]:
df = pd.get_dummies(df)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB,BernoulliNB
from sklearn.svm import LinearSVC, SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
%matplotlib inline

In [ ]:
#Linear regression, first create test and train dataset
x=df.loc[:,['X','OBJECTID','BLOCK_X','BLOCK_Y','HEIGHTNUM', 'WEIGHTNUM','PSA']].values
y=df.loc[:,'Y'].values

In [ ]:
# Creating a test and training dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [ ]:
# Linear regression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
print(regressor.intercept_)
print(regressor.coef_)

In [ ]:
#Displaying the difference between the actual and the predicted
y_pred = regressor.predict(X_test)
df_output = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(df_output)

In [ ]:
#Checking the accuracy of Linear Regression
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(y_pred,y_test,color='black',linestyle='dashed',marker='*',markerfacecolor='red',markersize=10)
plt.title('Error analysis')
plt.xlabel('Predicted values')
plt.ylabel('Test values')

In [ ]:
# Cross validation score
x=df.loc[:,['X','OBJECTID']].values
y=df.loc[:,'Y'].values
regressor=LinearRegression()
mse=cross_val_score(regressor,X_train,y_train,scoring='neg_mean_squared_error',cv=5)
mse_mean=np.mean(mse)
print(mse_mean)
diff=metrics.mean_squared_error(y_test, y_pred)-abs(mse_mean)
print(diff)

In [ ]:
x=df.loc[:,['X']].values
y=df.loc[:,'Y'].values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 1)

#Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(x_train)
x_train=sc.transform(x_train)
x_test=sc.transform(x_test)

In [ ]:
# KNN Classification
# sorted(sklearn.neighbors.VALID_METRICS['brute'])
knn = KNeighborsClassifier(n_neighbors = 17)
knn.fit(x_train,y_train)
y_pred=knn.predict(x_test)

Since I got ValueError: Unknown label type: 'continuous' I won't perform the rest of the script.

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Be patient. Marília Prata, @mpwolke was Here.' )